In [5]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F

from einops import rearrange
#from torchsummary import summary

import sys
sys.path.insert(0, '../helpers')
# sys.path.insert(0,'/home/labs/rudich/sagima/helpers')
from plotting import *
from validate import *
from models import *
from data_handlers import *
from training import *
from metrics import *
from explore import *
from torchsummary import *
# from visual_helpers_2 import *

#from visual_helpers import *

import itertools
import json


#taken from the model building section - 
import pandas as pd
pd.set_option('max_colwidth', 800)


import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
# import xgboost as xg
from statsmodels.tools.eval_measures import rmse
import scipy 
from statsmodels.tsa.stattools import grangercausalitytests
import torch.optim as optim
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.fft import fft, fftfreq
import folium

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.colors import Normalize
from matplotlib.patches import Rectangle
import cartopy.feature


# import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio


import ast
from scipy.spatial.distance import cdist

from tqdm import tqdm


In [6]:
df_cord = pd.read_csv("/home/labs/rudich/sagima/TOAR/coordinates_pm.csv")
df_cord = df_cord.rename(columns={df_cord.columns[0]:'station_id'})
df_cord

,station_id,lat,lng,alt
0,4997,37.311211,27.782933,-999.0
1,5132,37.030961,27.971696,-999.0
2,5134,41.535830,20.698900,-999.0
3,5135,42.135560,21.714740,-999.0
4,5136,42.003610,21.463620,-999.0
...,...,...,...,...
652,57645,42.533000,20.528000,500.0
653,57646,42.187769,21.030861,1803.0
654,72429,40.622749,40.306088,-999.0
655,72510,36.302961,33.918144,-999.0


In [7]:
df_cord = pd.read_csv("/home/labs/rudich/sagima/TOAR/coordinates_pm.csv")
df_cord = df_cord.rename(columns={df_cord.columns[0]:'station_id'})
df_cord

df_pm0 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_0.csv",index_col=0).dropna(axis=1,how='all')
df_pm1 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_1.csv",index_col=0).dropna(axis=1,how='all')
df_pm2 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_2.csv",index_col=0).dropna(axis=1,how='all')
df_pm3 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_3.csv",index_col=0).dropna(axis=1,how='all')
df_pm4 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_4.csv",index_col=0).dropna(axis=1,how='all')
df_pm5 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_5.csv",index_col=0).dropna(axis=1,how='all')
df_pm6 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_6.csv",index_col=0).dropna(axis=1,how='all')

df_pm =(
        pd.merge(df_pm0, df_pm1, how='outer',left_index=True, right_index=True)
        .pipe(lambda d:pd.merge(d, df_pm2, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm3, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm4, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm5, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm6, how='outer',left_index=True, right_index=True))
)

In [8]:
def convert_string_to_tuple_list(string_list):
    tuple_list = []
    for string in string_list:
        # Remove leading and trailing whitespace and parentheses
        string = string.strip('()')
        # Convert string to tuple using ast.literal_eval
        tuple_value = ast.literal_eval(string)
        tuple_list.append(tuple_value)
    return tuple_list

with open("saved_turkey_lat_lon_cords", "rb") as fp:   # Unpickling
    saved_turkey_lat_lon_cords = pickle.load(fp)

with open("saved_turkey_lat_lon_idx", "rb") as fp:   # Unpickling
    saved_turkey_lat_lon_idx = pickle.load(fp)
    
with open("saved_greece_lat_lon_cords", "rb") as fp:   # Unpickling
    saved_greece_lat_lon_cords= pickle.load(fp)

with open("saved_greece_lat_lon_idx", "rb") as fp:   # Unpickling
    saved_greece_lat_lon_idx = pickle.load(fp)
    
saved_turkey_lat_lon_cords = convert_string_to_tuple_list(saved_turkey_lat_lon_cords)
saved_turkey_lat_lon_cords.append((35.0,27.0))
saved_turkey_lat_lon_cords.append((35.0, 45.5))

with open("saved_turkey_and_greece_lat_lon_cords", "rb") as fp:   # Unpickling
    saved_turkey_and_greece_lat_lon_cords= pickle.load(fp)

with open("saved_turkey_and_greece_lat_lon_idx", "rb") as fp:   # Unpickling
    saved_turkey_and_greece_lat_lon_idx = pickle.load(fp)


In [9]:
#ls /home/labs/rudich/Collaboration/dust_prediction/data/interpolated_mean_normalized/

# Dnote:
# MPM - maps of PM field (nXhXW)
# PPM - pixels satellite (n X #pixels )
# MPPM - average pixels satellite (n vector)
# SPM - average over 30 stations (n vector) 


colab_dir = "/home/labs/rudich/Collaboration/dust_prediction/data"
meteo_dir = colab_dir+"/interpolated_mean_normalized/"
meteo_de_dir = colab_dir+"/meteorology_renormalization_tensors"
dust_dir = colab_dir+"/dust_61368_108_2_339/"

pm10_all = torch.load(meteo_de_dir+"/meteorology_tensors_1_81_189_interpolated_mean_normalized_pm10_tensor_denormalized.pkl")
labels_all = torch.load(dust_dir+"dust_61368_108_2_339_full_tensor.pkl")

timestamps_labels = torch.load(dust_dir+"dust_61368_108_2_339_full_timestamps.pkl")
timestamps_mpm = pd.to_datetime(torch.load(colab_dir+"/meteorology_tensors_1_81_189_general_timestamps.pkl"))
timestamps_all = pd.to_datetime(torch.load(colab_dir+"/meteorology_tensors_1_81_189_general_timestamps.pkl"))




In [10]:
colab_dir = "/home/labs/rudich/Collaboration/dust_prediction/data"
meteo_dir = colab_dir+"/interpolated_mean_normalized/"
meteo_de_dir = colab_dir+"/meteorology_renormalization_tensors"
dust_dir = colab_dir+"/dust_61368_108_2_339/"

meteo_meta = torch.load(meteo_dir+"/meteorology_tensor_62_81_189_metadata.pkl")
dust_meta = torch.load(dust_dir+"/dust_61368_108_2_339_metadata.pkl")

lons_dict = meteo_meta['dims']['lons']
lats_dict = meteo_meta['dims']['lats']



In [11]:
df_pm0 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_0.csv",index_col=0).dropna(axis=1,how='all')
df_pm1 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_1.csv",index_col=0).dropna(axis=1,how='all')
df_pm2 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_2.csv",index_col=0).dropna(axis=1,how='all')
df_pm3 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_3.csv",index_col=0).dropna(axis=1,how='all')
df_pm4 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_4.csv",index_col=0).dropna(axis=1,how='all')
df_pm5 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_5.csv",index_col=0).dropna(axis=1,how='all')
df_pm6 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_6.csv",index_col=0).dropna(axis=1,how='all')

df_pm =(
        pd.merge(df_pm0, df_pm1, how='outer',left_index=True, right_index=True)
        .pipe(lambda d:pd.merge(d, df_pm2, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm3, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm4, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm5, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm6, how='outer',left_index=True, right_index=True))


)
df_cord = pd.read_csv("/home/labs/rudich/sagima/TOAR/coordinates_pm.csv")
df_cord = df_cord.rename(columns={df_cord.columns[0]:'station_id'})


first_year = int(df_pm.index[0][:4])
last_year = int(df_pm.index[-1][:4])

timestamps = df_pm.index.values.astype('datetime64[s]').astype(np.int64)
stn_ids = df_pm.columns.values.astype(np.int64)


pm10_values = df_pm.to_numpy()

pm10_tensor = pm10_values.reshape(len(timestamps), len(stn_ids), 1)

# Convert the tensor to a PyTorch tensor
labels_all = torch.from_numpy(pm10_tensor)

timestamp_labels = pd.to_datetime(df_pm.index)

zero_var = (pm10_all.var((2,3))==0).reshape(-1).numpy()
pm10_valid_timestamps = timestamps_all[~zero_var]
pm10_valid_all = pm10_all[~zero_var]

l = [pm10_valid_timestamps, timestamp_labels]
shared_timestamps = l[0].intersection(l[1])

shared_timestamps_idxs = []
for timestamps_list in tqdm(l):
    shared_idxs_per_list = []
    for shared_t in shared_timestamps:
        try:
            shared_idxs_per_list.append(timestamps_list.get_loc(shared_t))
            
        except:
            print(f"Error! Something wierd happend with {shared_t} and {timestamps_list}. Aborting")
        if shared_t == shared_timestamps[-1]:
            print(max(shared_idxs_per_list))
    shared_timestamps_idxs.append(shared_idxs_per_list)

inputs_all = pm10_valid_all
labels_all = labels_all



years = [ i for i in range(first_year, last_year)]
months = [i for i in range(1,13)]

dt = pd.Series(shared_timestamps).dt
years = np.isin(dt.year, years)
months = np.isin(dt.month, months)
include = (years) * (months)

pm10_shared_timestamps = shared_timestamps[include]
inputs = inputs_all[shared_timestamps_idxs[0]][include]
labels = labels_all[shared_timestamps_idxs[1]][include]



/tmp/ipykernel_58224/2475775844.py:26: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  timestamps = df_pm.index.values.astype('datetime64[s]').astype(np.int64)
 50%|███████████████████████████████████████████████████                                                   | 1/2 [00:00<00:00,  1.32it/s]

52513


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.51it/s]

61367


In [12]:
channels46 = ["Q250","Q500","Q700","Q850","Q900",
            "T250","T500","T700","T850","T900",
            "U250","U500","U700","U850","U900",
            "V250","V500","V700","V850","V900",
            "Z250","Z500","Z700","Z850","Z900",
            "PV250","PV500","PV700","PV850","PV900",
            "OMEGA250","OMEGA500","OMEGA700","OMEGA850","OMEGA900",
            "SLP","u10","v10","duaod550",'aermr06_50','aermr06_40','aermr06_30', 'aermr06_20',
              "pm2p5","pm10","tcwv"]


In [19]:

tensors_list = []
pixels_list = []
#labels_all = torch.load(dust_dir+"dust_61368_108_2_339_full_tensor.pkl")
for channel in channels_for_vis:
    data_dir = "/home/labs/rudich/Collaboration/dust_prediction/data/interpolated_mean_normalized/"
    dust_dir = "/home/labs/rudich/Collaboration/dust_prediction/data/dust_61368_108_2_339/"
    channel_all = torch.load(f"/home/labs/rudich/Collaboration/dust_prediction/data/meteorology_renormalization_tensors/meteorology_tensors_1_81_189_interpolated_mean_normalized_{channel}_tensor_denormalized.pkl")
    tensors_list.append(channel_all)
stacked_tensor = torch.stack(tensors_list, dim=1)
stacked_tensor = stacked_tensor.squeeze(dim=2)
stacked_tensor = stacked_tensor[~zero_var]
print(stacked_tensor.shape)

torch.Size([52514, 4, 81, 189])


In [17]:

tensors_list = []
pixels_list = []
#labels_all = torch.load(dust_dir+"dust_61368_108_2_339_full_tensor.pkl")
for channel in tqdm(range(len(channels46))):
    data_dir = "/home/labs/rudich/Collaboration/dust_prediction/data/interpolated_mean_normalized/"
    dust_dir = "/home/labs/rudich/Collaboration/dust_prediction/data/dust_61368_108_2_339/"
    channel_all = torch.load(f"/home/labs/rudich/Collaboration/dust_prediction/data/meteorology_renormalization_tensors/meteorology_tensors_1_81_189_interpolated_mean_normalized_{channels46[channel]}_tensor_denormalized.pkl")
    tensors_list.append(channel_all)
stacked_tensor = torch.stack(tensors_list, dim=1)
stacked_tensor = stacked_tensor.squeeze(dim=2)
stacked_tensor = stacked_tensor[~zero_var]
print(stacked_tensor.shape)

 17%|█████████████████▌                                                                                   | 8/46 [00:27<02:12,  3.50s/it]


KeyboardInterrupt: 

In [25]:
# Define the slice indices
lat_start, lat_end =  54, 66
lon_start, lon_end = 104, 117

sliced_tensor = stacked_tensor[:, :, lat_start:lat_end, lon_start:lon_end]

# Create new dictionaries for the sliced lat and lon indices
new_lats_dict = {i: lats_dict[i] for i in range(lat_start, lat_end)}
new_lons_dict = {i: lons_dict[i] for i in range(lon_start, lon_end)}

def replace_keys_with_sequential_integers(dictionary):
    new_dict = {}
    keys = list(dictionary.keys())
    for i in range(len(keys)):
        new_dict[i] = dictionary[keys[i]]
    return new_dict

new_lats_dict = replace_keys_with_sequential_integers(new_lats_dict)
new_lons_dict = replace_keys_with_sequential_integers(new_lons_dict)

del stacked_tensor


In [7]:
all_stations_lonlat = (df_cord 
                       .rename(columns={'lng':'lon'})
                       .assign(stn_id=np.arange(0,df_cord.shape[0],1))
                        )


In [8]:
def create_matrix(list1, list2):
    matrix = [[(num1, num2) for num2 in list2] for num1 in list1]
    return matrix

def calculate_distances(matrix, df):
    lat1 = matrix[:, 0].reshape(-1, 1)
    lon1 = matrix[:, 1].reshape(-1, 1)

    lat2 = df['lat'].values.reshape(1, -1)
    lon2 = df['lon'].values.reshape(1, -1)

    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    radius = 6371.0

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    dlat_sq = np.sin(dlat / 2) ** 2
    dlon_sq = np.sin(dlon / 2) ** 2

    a = dlat_sq + np.cos(lat1_rad) * np.cos(lat2_rad) * dlon_sq
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distances = radius * c

    return distances

station_df = all_stations_lonlat
pm10_df = df_pm

averaged_values = {}


matrix = create_matrix(list(lats_dict.values()), list(lons_dict.values()))
matrix = np.array(matrix).reshape(-1, 2)

In [9]:

stations_tensor = np.zeros((pm10_valid_timestamps.shape[0], 1, len(lats_dict), len(lons_dict)))

distances = cdist(matrix, station_df[['lat', 'lon']], metric='euclidean')
result = calculate_distances(matrix, station_df[['lat', 'lon']])

inv_lats = {v: k for k, v in lats_dict.items()}
inv_lons = {v: k for k, v in lons_dict.items()}
list_of_pixels_with_stations = []

In [10]:
pm10_df.index = pd.to_datetime(pm10_df.index)
pm10_df.columns =  pm10_df.columns.astype(int)
set1 = set(pm10_valid_timestamps)
set2 = set(pm10_df.index)

# Find the intersection of the sets
intersection = set1.intersection(set2)

# Convert the intersection set to a DatetimeIndex
common_timestamps = pd.DatetimeIndex(pd.to_datetime(list(intersection)))
common_timestamps = common_timestamps.sort_values()
# Find the index where the intersection starts
start_index = list(pm10_valid_timestamps).index(common_timestamps[0])

# Find the index where the intersection ends
end_index = list(pm10_valid_timestamps).index(common_timestamps[-1])


In [16]:
# common_timestamps.to_frame().to_csv('common_timestamps_df.csv')

# Creating a dict with pixel coordinates - stations id - distances 

In [11]:
threshold = 56
pixels_to_stations_dict = {}
pixels_to_stations_dict2 = {}
for i in tqdm(range(distances.shape[0])):
    nearby_stations = station_df[np.abs(result[i]) < threshold] #number taken using a test for the distance between the pixels
    nearby_distances = result[i][np.abs(result[i]) < threshold]
    if len(nearby_stations) > 0:

        list_of_pixels_with_stations.append(matrix[i])
        str_list_lat_lon = [str(i) for i in nearby_stations[['lat','lon']].values.tolist()]
        str_list_id = nearby_stations['station_id'].values.tolist()
        pixels_to_stations_dict[str(matrix[i])] = dict(zip(str_list_lat_lon, list(nearby_distances)))
        pixels_to_stations_dict2[str(matrix[i])] = dict(zip(str_list_id, list(nearby_distances)))

simple_dict = {key: list(inner_dict.keys()) for key, inner_dict in pixels_to_stations_dict.items()}

100%|███████████████████████████████████████████████████████████████████████████| 15309/15309 [00:02<00:00, 7171.22it/s]


In [13]:
valid_stations = df_pm.columns.tolist() #stations with data on them
def filter_dict_by_list(dictionary, values_list):
    # Create a new dictionary to store the filtered values
    filtered_dict = {}

    # Iterate over the dictionary items
    for key, value_list in dictionary.items():
        # Filter the list based on the values_list
        filtered_list = [value for value in value_list if value in values_list]
        
        # Add the key-filtered_list pair to the filtered dictionary
        filtered_dict[key] = filtered_list

    return filtered_dict

simple_dict = filter_dict_by_list(simple_dict,valid_stations)

In [15]:
# import pickle
# with open('pixel_to_station_id.pkl','wb' ) as f:
#     pickle.dump(simple_dict, f)

In [110]:
stations_tensor[:,0,:,:] = np.nan
# Iterate over each key in the simple_dict
for key in tqdm(simple_dict):
    # Extract the latitude and longitude values from the key
    values = key[1:-1].split()

    # Convert the values to float
    values = [float(value) for value in values]

    lat = values[0]
    lon = values[1]

    # Get the corresponding indices from the inv_lats and inv_lons dictionaries
    lat_index = inv_lats[lat]
    lon_index = inv_lons[lon]

    # Iterate over each timestamp in the common_timestamps list
    for t, timestamp in enumerate(common_timestamps):
        if t >= start_index:
            # Get the corresponding row in the pm10_df DataFrame based on the timestamp
            df_row = pm10_df.loc[timestamp]

            # Filter the columns in the DataFrame based on the values from the simple_dict
            filtered_df = df_row[simple_dict[key]]

            # Calculate the average value of the filtered columns
            mean_value = filtered_df.mean()

            # Update the tensor at the specified indices with the mean value
            stations_tensor[t, 0, lat_index, lon_index] = mean_value


100%|████████████████████████████████████████████████████████████████████████████████| 187/187 [1:36:49<00:00, 31.07s/it]


In [124]:

# Define the slice indices
lat_start, lat_end = 50, 65
lon_start, lon_end = 103, 154

# Slice the tensor
sliced_tensor = full[:, :, lat_start:lat_end, lon_start:lon_end]

# Create new dictionaries for the sliced lat and lon indices
new_lats_dict = {i: lats_dict[i] for i in range(lat_start, lat_end)}
new_lons_dict = {i: lons_dict[i] for i in range(lon_start, lon_end)}

def replace_keys_with_sequential_integers(dictionary):
    new_dict = {}
    keys = list(dictionary.keys())
    for i in range(len(keys)):
        new_dict[i] = dictionary[keys[i]]
    return new_dict

new_lats_dict = replace_keys_with_sequential_integers(new_lats_dict)
new_lons_dict = replace_keys_with_sequential_integers(new_lons_dict)

del full



# tensor to dataframe

In [41]:

tensor = sliced_tensor
# Given information
tensor_shape = (52514, 47, 15, 51)

timestamps = common_timestamps # List of timestamps (length 52514)

# Reshape the tensor
reshaped_tensor = np.reshape(tensor, (tensor_shape[0], tensor_shape[1], -1))

# Initialize an empty list to store individual DataFrames
dfs = []

# Iterate over lat and lon combinations
for lat_index in tqdm(range(tensor_shape[2])):
    for lon_index in range(tensor_shape[3]):        
        # Create a DataFrame for the current lat-lon combination
        df = pd.DataFrame(reshaped_tensor[:, :, (lat_index * tensor_shape[3]) + lon_index])

        df['time'] = timestamps
        lat_loc = new_lats_dict[str(lat_index)]
        lon_loc = new_lons_dict[str(lon_index)]
        df['lat_lon_coord'] = str((float(lat_loc),float(lon_loc)))

        dfs.append(df)

# Concatenate all the individual DataFrames into one big DataFrame
big_df = pd.concat(dfs, ignore_index=True)

df = (big_df.set_axis(channels46 +['stn_mesm','time','lat_lon_coord'],axis=1 )
      .drop('pm2p5',axis=1)
     )

100%|████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 20.47it/s]


,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,time,lat_lon_coord
0,0.000011,0.000138,0.002325,0.004684,0.005909,221.524216,252.856476,267.699463,277.711273,280.923279,...,-7.709136e-11,-2.025620e-12,1.387664e-11,3.461629e-11,1.464825e-08,2.722577e-08,17.485289,NaN,2003-01-01 00:00:00+00:00,"(35.0, 19.5)"
1,0.000011,0.000337,0.001836,0.004204,0.005016,222.469406,248.434280,267.991730,277.618134,280.901459,...,-3.875626e-11,-1.081209e-12,1.330524e-11,3.313251e-11,1.773758e-08,3.374522e-08,15.694367,NaN,2003-01-01 03:00:00+00:00,"(35.0, 19.5)"
2,0.000010,0.000242,0.000516,0.003380,0.005039,222.595139,247.227905,268.481232,277.179199,280.650970,...,-4.211665e-13,-1.367978e-13,1.273384e-11,3.164874e-11,2.082692e-08,4.026467e-08,13.903446,NaN,2003-01-01 06:00:00+00:00,"(35.0, 19.5)"
3,0.000008,0.000229,0.001980,0.004509,0.005848,223.250992,248.208664,266.922791,277.502838,280.649384,...,1.237930e-12,-6.595037e-14,4.534037e-11,3.155716e-11,2.201033e-08,4.246103e-08,14.509666,NaN,2003-01-01 09:00:00+00:00,"(35.0, 19.5)"
4,0.000009,0.000145,0.001721,0.004269,0.005898,221.955292,248.784119,267.021027,277.123169,280.224884,...,2.897027e-12,4.897048e-15,7.794691e-11,3.146559e-11,2.319373e-08,4.465739e-08,15.115888,NaN,2003-01-01 12:00:00+00:00,"(35.0, 19.5)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40173205,0.000022,0.000141,0.000723,0.002411,0.002335,218.902679,257.543121,273.394135,276.424622,279.130249,...,3.326063e-09,2.786967e-11,5.412007e-15,0.000000e+00,1.153348e-08,1.580973e-08,2.493623,NaN,2020-12-31 09:00:00+00:00,"(42.0, 44.5)"
40173206,0.000024,0.000139,0.000756,0.002895,0.002766,218.872070,258.164948,273.569397,276.014679,278.239471,...,2.763189e-09,3.063281e-11,6.874656e-15,0.000000e+00,1.103926e-08,1.543707e-08,2.518091,NaN,2020-12-31 12:00:00+00:00,"(42.0, 44.5)"
40173207,0.000025,0.000139,0.000752,0.002678,0.003193,218.995911,257.895905,273.583618,276.744629,278.258575,...,2.604069e-09,3.633998e-11,3.397406e-15,0.000000e+00,1.048309e-08,1.430515e-08,2.394837,NaN,2020-12-31 15:00:00+00:00,"(42.0, 44.5)"
40173208,0.000024,0.000142,0.000845,0.002460,0.003452,219.082367,257.514435,273.437439,277.136749,277.807831,...,2.444948e-09,4.204716e-11,-7.984380e-17,0.000000e+00,9.926912e-09,1.317322e-08,2.271582,NaN,2020-12-31 18:00:00+00:00,"(42.0, 44.5)"


In [51]:
grouped = df.groupby('lat_lon_coord')

# Creating new columns with previous row values for each group
for column in df.drop(['stn_mesm','time','lat_lon_coord'],axis=1).columns:
    new_column_name = column + '_prev'
    df[new_column_name] = grouped[column].shift()
    
df['week'] = df['time'].dt.strftime('%W')
df['hour'] = df['time'].dt.hour
df['month'] = df['time'].dt.month
df['year'] = df['time'].dt.year
df['sin'] = np.sin(df['time'].dt.dayofyear * 2 * np.pi / 365)
df['cos'] = np.cos(df['time'].dt.dayofyear * 2 * np.pi / 365)



In [52]:
def add_delta_columns(df):
    channels_used =  ["Q250","Q500","Q700","Q850","Q900",
            "T250","T500","T700","T850","T900",
            "U250","U500","U700","U850","U900",
            "V250","V500","V700","V850","V900",
            "Z250","Z500","Z700","Z850","Z900",
            "PV250","PV500","PV700","PV850","PV900",
            "OMEGA250","OMEGA500","OMEGA700","OMEGA850","OMEGA900",
            "SLP","u10","v10","duaod550",'aermr06_50','aermr06_40','aermr06_30', 'aermr06_20',
              "tcwv"]

    # Loop over the channels names
    for col in channels_used:
        
        # Calculate the delta between the column and its corresponding "_prev" column
        delta_col = col + '_delta'
        prev_col = col + '_prev'
        df[delta_col] = df[col] - df[prev_col]
    
    return df

df = add_delta_columns(df)
df = df.drop(['pm10_prev'],axis=1)
df['target'] = df['stn_mesm'] - df['pm10']